In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers

In [2]:
! pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.9 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for einops from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.0 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.2-cp310-cp310-linux_x86_64.whl size=121773153 sha256=2818f8d82b162863d3dd00fb2ea5c4ef7d7361b04413a4f30dffe00f263da179
  Stored in directory: /root/.cache/pip/wheels/ff/8e/8b/5fb0f8eb882c58b2bcfb4302860884d1b45d8513f8b3daac9c
Successfully built flash-attn


In [3]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

model_path="/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer=AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    trust_remote_code = True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [14]:
%%time

def generate_essay(prompt):
    messages = [{
        "role":"user",
        "content": prompt
    }]

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors = "pt").to('cuda')
    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs,
            max_new_tokens = 7500,
            do_sample = True,
            pad_token_id = tokenizer.eos_token_id
        )

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    text = decoded[0].split("[/INST]")[1]
    return text

prompt = ''' '''

text = generate_essay(prompt)
text

CPU times: user 36.1 s, sys: 21 ms, total: 36.2 s
Wall time: 36.2 s


" The Mahabharata is one of the most famous and widely read epic poems of ancient India, with origins dating back to the 1st millennium BCE. It tells the story of a great war between two families, the Pandavas and the Kauravas, and follows the lives of a large cast of characters, including the gods, demons, and other mythical beings of Hinduism.\n\nThe Mahabharata is traditionally attributed to the sage Vyasa, who is believed to have collected and compiled the story from various sources, including other poets and seers. The epic is written in Sanskrit and is divided into 18 books, which explore themes of duty, honor, love, honor, loyalty, morality, righteousness, and the importance of spirituality in one's life.\n\nOne of the central themes of the Mahabharata is the conflict between right and wrong, and the consequences of one's actions. The Pandavas, who are righteous and virtuous, must confront the Kauravas and their allies, who are tyrannical and oppressive. This is a metaphor for t